In [2]:
!pip install paddlepaddle paddleocr
!python3 -m pip install paddlepaddle-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 MB 12.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.7/544.7 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.9/161.9 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 75.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 84.2 MB/s eta 0:00:00:00:010:01
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114248 sha256=08f21ea0d3b6bed576272d3b218e5ce81c25979a6cb176750cfe9d53633abae0
  Stored in directory: /root/.cache/pip/wheels/19/39/2f/2d3cadc408a8804103f1c34ddd4b9f6a93497b11fa96fe738e
Successfully built fire
  Attempting u

In [3]:
import paddle
print(paddle.utils.run_check())

Running verify PaddlePaddle program ... 


I1112 06:26:11.451818    30 program_interpreter.cc:212] New Executor is Running.
W1112 06:26:11.452292    30 gpu_resources.cc:119] Please NOTE: device: 0, GPU Compute Capability: 7.5, Driver API Version: 12.4, Runtime API Version: 11.8
W1112 06:26:11.464747    30 gpu_resources.cc:164] device: 0, cuDNN Version: 9.0.


PaddlePaddle works well on 1 GPU.


I1112 06:26:11.991675    30 interpreter_util.cc:624] Standalone Executor is Used.
======================= Modified FLAGS detected =======================
FLAGS(name='FLAGS_selected_gpus', current_value='1', default_value='')
I1112 06:26:13.968545   114 tcp_utils.cc:107] Retry to connect to 127.0.0.1:40489 while the server is not yet listening.
======================= Modified FLAGS detected =======================
FLAGS(name='FLAGS_selected_gpus', current_value='0', default_value='')
I1112 06:26:13.979517   113 tcp_utils.cc:181] The server starts to listen on IP_ANY:40489
I1112 06:26:13.979769   113 tcp_utils.cc:130] Successfully connected to 127.0.0.1:40489
I1112 06:26:16.968828   114 tcp_utils.cc:130] Successfully connected to 127.0.0.1:40489
I1112 06:26:17.008651   114 process_group_nccl.cc:129] ProcessGroupNCCL pg_timeout_ 1800000
W1112 06:26:17.011770   114 gpu_resources.cc:119] Please NOTE: device: 1, GPU Compute Capability: 7.5, Driver API Version: 12.4, Runtime API Version: 11.

PaddlePaddle works well on 2 GPUs.
PaddlePaddle is installed successfully! Let's start deep learning with PaddlePaddle now.
None


I1112 06:26:18.514158   131 tcp_store.cc:289] receive shutdown event and so quit from MasterDaemon run loop


In [4]:
!pip install torch_xla==2.5.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 MB 18.0 MB/s eta 0:00:00:00:0100:01


In [ ]:
import pandas as pd
file_path = '/kaggle/input/amazon-ml-challenge-2024/train.csv'
df = pd.read_csv(file_path)
df.iloc[150000:160000].entity_value

In [ ]:
# Importing necessary libraries
import pandas as pd
import requests
import cv2
import numpy as np
from io import BytesIO
from paddleocr import PaddleOCR

# Initialize PaddleOCR
ocr = PaddleOCR(use_angle_cls=True, lang='en', use_gpu=True)

# Load the CSV file
file_path = '/kaggle/input/amazon-ml-challenge-2024/train.csv'
df = pd.read_csv(file_path)

# Function to download image and perform OCR using OpenCV
def extract_text_from_image(url):
    try:
        # Download the image
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Check if the request was successful
        
        # Convert image content to NumPy array for OpenCV
        img_array = np.asarray(bytearray(response.content), dtype=np.uint8)
        img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)  # Load the image using OpenCV
        
        # If image is not loaded properly, log an error
        if img is None:
            print(f"Error loading image: {url}")
            return None
        
        # Use PaddleOCR to extract text
        result = ocr.ocr(img)
        
        # Collect the text from OCR result
        extracted_text = " ".join([line[1][0] for line in result[0]])
        return extracted_text

    except requests.exceptions.RequestException as req_err:
        print(f"Request error processing {url}: {req_err}")
        return None

    except Exception as e:
        print(f"Unexpected error processing {url}: {e}")
        return None

# Iterate over the DataFrame and process each image in batches
batch_size = 1000  # Save progress after every 1000 iterations
max_images = 5000  # Maximum number of images to process from the starting point (50,001 to 100,000)
start_index = 155000  # Starting index
end_index = 160000  # Ending index to avoid out-of-bounds error
processed_images = 0  # Track the number of processed images

# Adjust the loop to start from 50001 and end at 100000
for i in range(start_index, end_index, batch_size):
    batch_df = df.iloc[i:i+batch_size]  # Process images in batches
    batch_df['extracted_text'] = batch_df['image_link'].apply(extract_text_from_image)
    
    # Save the updated DataFrame to a CSV file after every batch
    if processed_images == 0:  # For the first batch
        batch_df.to_csv('test_temp_2.csv', mode='w', index=False, header=True)
    else:
        batch_df.to_csv('test_temp_2.csv', mode='a', index=False, header=False)

    processed_images += len(batch_df)  # Update the number of processed images
    print(f"Processed {processed_images} entries")

print("Processing complete!")


download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/en_PP-OCRv3_det_infer.tar


100%|██████████| 3910/3910 [00:04<00:00, 858.63it/s] 


download https://paddleocr.bj.bcebos.com/PP-OCRv4/english/en_PP-OCRv4_rec_infer.tar to /root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer/en_PP-OCRv4_rec_infer.tar


100%|██████████| 10000/10000 [00:18<00:00, 528.00it/s]


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2138/2138 [00:15<00:00, 134.60it/s]

[2024/11/12 04:41:51] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=True, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_ch

[2024/11/12 04:41:52] ppocr WARNING: The first GPU is used for inference by default, GPU ID: 0
[2024/11/12 04:41:53] ppocr WARNING: The first GPU is used for inference by default, GPU ID: 0
[2024/11/12 04:41:55] ppocr DEBUG: dt_boxes num : 11, elapsed : 0.6769497394561768
[2024/11/12 04:41:55] ppocr DEBUG: cls num  : 11, elapsed : 0.06569433212280273
[2024/11/12 04:41:55] ppocr DEBUG: rec_res num  : 11, elapsed : 0.11088109016418457
[2024/11/12 04:41:55] ppocr DEBUG: dt_boxes num : 11, elapsed : 0.06251811981201172
[2024/11/12 04:41:55] ppocr DEBUG: cls num  : 11, elapsed : 0.016659259796142578
[2024/11/12 04:41:56] ppocr DEBUG: rec_res num  : 11, elapsed : 0.03338789939880371
[2024/11/12 04:41:56] ppocr DEBUG: dt_boxes num : 7, elapsed : 0.055979251861572266
[2024/11/12 04:41:56] ppocr DEBUG: cls num  : 7, elapsed : 0.026380538940429688
[2024/11/12 04:41:56] ppocr DEBUG: rec_res num  : 7, elapsed : 0.04482722282409668
[2024/11/12 04:41:56] ppocr DEBUG: dt_boxes num : 2, elapsed : 0.05

In [5]:
print("hello world")

hello world


In [6]:
import pandas as pd

# Load the CSV file with the extracted text
csv_file = '/kaggle/input/10kimages/merged_file.csv'
df = pd.read_csv(csv_file)

# Drop rows where the 'extracted_text' field is empty or NaN
df_cleaned = df[df['extracted_text'].notna() & df['extracted_text'].str.strip().astype(bool)]

# Save the cleaned DataFrame to a new CSV file
df_cleaned.to_csv('cleaned_train_with_text.csv', index=False)

print(f"Rows with non-empty 'extracted_text' field saved to 'cleaned_train_with_text.csv'.")

Rows with non-empty 'extracted_text' field saved to 'cleaned_train_with_text.csv'.


In [4]:
!pip install transformers torch pandas


In [7]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel

csv_file = '/kaggle/working/cleaned_train_with_text.csv'
df = pd.read_csv(csv_file)

model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    cls_embedding = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
    return cls_embedding

df['bert_embedding'] = df['extracted_text'].apply(lambda x: get_bert_embedding(x) if isinstance(x, str) else None)

df.to_csv('bert_embeddings_train_with_text.csv', index=False)

print(f"BERT embeddings added and saved to 'bert_embeddings_train_with_text.csv'.")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

cuda
BERT embeddings added and saved to 'bert_embeddings_train_with_text.csv'.


In [8]:
import pandas as pd

csv_file = '/kaggle/working/bert_embeddings_train_with_text.csv'
df = pd.read_csv(csv_file)

# Extract the image name from the image_link field (last part after '/')
df['image_name'] = df['image_link'].apply(lambda x: x.split('/')[-1] if isinstance(x, str) else None)

# Save the DataFrame with the new image_name column
output_file = '/kaggle/working/train_with_image_name.csv'  # Save to working directory
df.to_csv(output_file, index=False)

print(f"Image names added and saved to '{output_file}'.")

# After running the code, you can download the CSV file from the 'Output' section of the Kaggle notebook interface.


Image names added and saved to '/kaggle/working/train_with_image_name.csv'.


In [9]:
# Install transformers if not already installed
!pip install transformers
import torch
from transformers import BertTokenizer, BertForSequenceClassification


In [ ]:
# pip install torch==2.0.1 torch_xla==2.0.1


In [10]:
from transformers import BartTokenizer, BartForSequenceClassification

# Specify the path where the model is saved
model_path = '/kaggle/input/bart-base/pytorch/default/1/junu'

# Load the BART tokenizer
tokenizer = BartTokenizer.from_pretrained(model_path)

# Load the BART model for sequence classification (or adjust for your task)
model = BartForSequenceClassification.from_pretrained(model_path)
model.eval()  # Set the model to evaluation mode


Some weights of BartForSequenceClassification were not initialized from the model checkpoint at /kaggle/input/bart-base/pytorch/default/1/junu and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BartForSequenceClassification(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_la

In [11]:
def preprocess(text, tokenizer, max_length=128):
    return tokenizer(text, padding='max_length', truncation=True, max_length=max_length, return_tensors="pt")


In [14]:
import pandas as pd

# Load the file
file_path = '/kaggle/working/train_with_image_name.csv'
data = pd.read_csv(file_path)

# Create the combined_text column
data['combined_text'] = data.apply(
    lambda row: f"{row['entity_value']} | {row['extracted_text']}", axis=1
)

# Save the updated file
output_path = '/kaggle/working//bert_embeddings_with_combined_text.csv'
data.to_csv(output_path, index=False)


In [15]:
import pandas as pd

csv_file = '/kaggle/working/bert_embeddings_with_combined_text.csv'
df = pd.read_csv(csv_file)



In [16]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

In [20]:
from transformers import BartTokenizer, BartForConditionalGeneration
from sklearn.model_selection import train_test_split

# Initialize the tokenizer and model
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

# Example data
X = df['combined_text']
y = df['entity_value']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=104, test_size=0.9, shuffle=True)

def tokenize_function(text_list):
    return tokenizer(text_list, return_tensors="pt", padding=True, truncation=True, max_length=512)

input_tokens = [tokenize_function(x)['input_ids'].squeeze(0) for x in X_train]
target_tokens = [tokenize_function(y)['input_ids'].squeeze(0) for y in y_train]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [21]:
import torch
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm

# Set the model to train mode
model.train()

# Send model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)  # Adjust learning rate if necessary

# Convert tokenized data to tensors and move to GPU
input_tokens = [x.to(device) for x in input_tokens]
target_tokens = [y.to(device) for y in target_tokens]

# Define a basic training loop
epochs = 3  # Adjust based on your needs
batch_size = 8  # Adjust batch size for performance

for epoch in range(epochs):
    total_loss = 0
    print(f'Epoch {epoch + 1}/{epochs}')

    # Create a tqdm progress bar for the batch loop
    for i in tqdm(range(0, len(input_tokens), batch_size), desc="Training Progress", unit="batch"):
        # Prepare inputs and targets batch with padding
        inputs_batch = pad_sequence(input_tokens[i:i + batch_size], batch_first=True, padding_value=tokenizer.pad_token_id)
        targets_batch = pad_sequence(target_tokens[i:i + batch_size], batch_first=True, padding_value=tokenizer.pad_token_id)

        optimizer.zero_grad()

        # Forward pass: get model predictions
        outputs = model(input_ids=inputs_batch, labels=targets_batch)

        # Loss is calculated automatically by model (cross-entropy)
        loss = outputs.loss
        total_loss += loss.item()

        # Backward pass: compute gradients and update weights
        loss.backward()
        optimizer.step()

    # Display the average loss for this epoch
    print(f'Epoch {epoch + 1} completed. Average Loss: {total_loss / len(input_tokens)}')

print("Training complete")

Epoch 1/3


Training Progress: 100%|██████████| 125/125 [00:14<00:00,  8.59batch/s]


Epoch 1 completed. Average Loss: 0.07561718242780449
Epoch 2/3


Training Progress: 100%|██████████| 125/125 [00:14<00:00,  8.50batch/s]


Epoch 2 completed. Average Loss: 0.003691294525379272
Epoch 3/3


Training Progress: 100%|██████████| 125/125 [00:14<00:00,  8.43batch/s]

Epoch 3 completed. Average Loss: 0.0013365371798758651
Training complete


In [22]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score

# Set the model to evaluation mode
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# Prepare test inputs
input_tokens_test = [
    tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)['input_ids'].squeeze(0).to(device)
    for text in X_test
]

# Initialize lists to store predicted and actual values
predicted_values = []
actual_values = list(y_test) 

# BLEU smoothing function
smooth_fn = SmoothingFunction().method4  # Handles cases where there is no overlap in n-grams

# Initialize variables for BLEU and accuracy
total_bleu_score = 0
correct_predictions = 0

# Lists to track exact match results for precision, recall, and F1 calculation
exact_match_results = []

# Loop through test data in batches
batch_size = 8
for i in tqdm(range(0, len(input_tokens_test), batch_size), desc="Evaluating"):
    inputs_batch = pad_sequence(
        input_tokens_test[i:i + batch_size], 
        batch_first=True, 
        padding_value=tokenizer.pad_token_id
    )

    # Generate predictions
    with torch.no_grad():
        generated_ids = model.generate(inputs_batch, max_length=20)

    # Decode predictions to text
    for idx, generated_id in enumerate(generated_ids):
        generated_text = tokenizer.decode(generated_id, skip_special_tokens=True)
        predicted_values.append(generated_text)

        # Calculate BLEU score for this prediction (reference is the actual value from y_test)
        actual_text = actual_values[i + idx]
        reference = [actual_text.split()]
        candidate = generated_text.split()

        # Calculate BLEU score for this sample
        bleu_score = sentence_bleu(reference, candidate, smoothing_function=smooth_fn)
        total_bleu_score += bleu_score

        # Exact match comparison
        is_exact_match = int(generated_text == actual_text)
        exact_match_results.append(is_exact_match)

# Compute average BLEU score
average_bleu_score = total_bleu_score / len(predicted_values)

# Calculate precision, recall, and F1 scores
precision = precision_score(exact_match_results, [1] * len(predicted_values), zero_division=0)
recall = recall_score(exact_match_results, [1] * len(predicted_values), zero_division=0)
f1 = f1_score(exact_match_results, [1] * len(predicted_values), zero_division=0)

# Print a few examples with BLEU scores
for i in range(10):
    print(f"Predicted: {predicted_values[i]}, Actual: {actual_values[i]}")
    reference = [actual_values[i].split()]
    candidate = predicted_values[i].split()
    sample_bleu = sentence_bleu(reference, candidate, smoothing_function=smooth_fn)
    print(f"BLEU score for sample {i}: {sample_bleu:.4f}")

# Print final evaluation metrics
print(f"\nAverage BLEU Score: {average_bleu_score:.4f}")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1 Score: {f1 * 100:.2f}%")


Evaluating: 100%|██████████| 1124/1124 [04:20<00:00,  4.31it/s]

Predicted: 6.69 inch, Actual: 6.69 inch
BLEU score for sample 0: 0.3211
Predicted: 16.5 centimetre, Actual: 16.5 centimetre
BLEU score for sample 1: 0.3211
Predicted: 5.5 centimetre, Actual: 5.5 centimetre
BLEU score for sample 2: 0.3211
Predicted: 5.0 centimetre, Actual: 5.0 centimetre
BLEU score for sample 3: 0.3211
Predicted: 21.7 inch, Actual: 21.7 inch
BLEU score for sample 4: 0.3211
Predicted: 24.0 inch, Actual: 24.0 inch
BLEU score for sample 5: 0.3211
Predicted: 15.0 inch, Actual: 15.0 inch
BLEU score for sample 6: 0.3211
Predicted: 6.0 centimetre, Actual: 6.0 centimetre
BLEU score for sample 7: 0.3211
Predicted: 23.6 centimetre, Actual: 23.6 centimetre
BLEU score for sample 8: 0.3211
Predicted: 30.0 millimetre, Actual: 30.0 millimetre
BLEU score for sample 9: 0.3211

Average BLEU Score: 0.3209
Precision: 99.94%
Recall: 100.00%
F1 Score: 99.97%
